In [12]:
import pandas as pd
features = pd.read_csv("../Data/reps.csv", header=None)
labels = pd.read_csv("../Data/labels.csv", header=None, names=["drop", "id"])
race_table = pd.read_csv("../Data/binned_id_gender_ethnicity.csv")

In [13]:
features.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.015975,0.061343,-0.169915,0.075664,0.080860,0.174134,-0.017157,-0.105687,0.071876,-0.029301,...,0.145569,0.129029,0.089958,-0.133423,-0.031034,-0.003381,-0.000501,0.095188,0.052924,0.061916
1,-0.083042,0.155064,-0.026929,-0.128313,0.002890,0.146170,-0.043166,0.020272,-0.146851,0.085454,...,0.108734,0.085527,0.075933,-0.095799,-0.008750,-0.023784,0.015153,0.029661,-0.078988,-0.013881
2,0.013870,0.185305,0.010482,-0.127374,0.080616,0.020505,0.014181,-0.103239,0.003175,0.028015,...,-0.102061,0.045702,0.023531,0.004467,-0.016138,0.034176,0.030275,0.086337,0.009568,-0.062744
3,0.044587,-0.058128,-0.112380,-0.075394,0.134542,0.067473,-0.025369,-0.002917,-0.155703,-0.118631,...,-0.012169,0.195434,-0.107325,0.021370,0.006799,0.036422,-0.091522,0.028600,-0.079282,-0.081262
4,0.066164,0.137578,-0.023866,0.052424,-0.012129,0.139718,-0.043587,-0.064435,-0.147255,0.022006,...,0.173739,0.143175,0.043278,-0.179890,-0.000041,-0.062348,0.074416,0.071871,0.201533,0.078662


In [31]:
labels['id'][1]
result.head()

,drop,id,0,1,2,3,4,5,6,7,...,118,119,120,121,122,123,124,125,126,127
0,1,4.284929e+10,-0.015975,0.061343,-0.169915,0.075664,0.080860,0.174134,-0.017157,-0.105687,...,0.145569,0.129029,0.089958,-0.133423,-0.031034,-0.003381,-0.000501,0.095188,0.052924,0.061916
1,1,6.623821e+10,-0.083042,0.155064,-0.026929,-0.128313,0.002890,0.146170,-0.043166,0.020272,...,0.108734,0.085527,0.075933,-0.095799,-0.008750,-0.023784,0.015153,0.029661,-0.078988,-0.013881
2,1,2.328813e+10,0.013870,0.185305,0.010482,-0.127374,0.080616,0.020505,0.014181,-0.103239,...,-0.102061,0.045702,0.023531,0.004467,-0.016138,0.034176,0.030275,0.086337,0.009568,-0.062744
3,1,7.569382e+10,0.044587,-0.058128,-0.112380,-0.075394,0.134542,0.067473,-0.025369,-0.002917,...,-0.012169,0.195434,-0.107325,0.021370,0.006799,0.036422,-0.091522,0.028600,-0.079282,-0.081262
4,1,5.623913e+10,0.066164,0.137578,-0.023866,0.052424,-0.012129,0.139718,-0.043587,-0.064435,...,0.173739,0.143175,0.043278,-0.179890,-0.000041,-0.062348,0.074416,0.071871,0.201533,0.078662


In [15]:
race_table.head()

,Unnamed: 0,id,gender,ethnicity
0,0,3211298559,1,caucasian
1,1,82733984485,1,caucasian
2,2,99029428429,0,caucasian
3,3,71174220620,1,caucasian
4,4,47408331317,1,caucasian


In [16]:
result = pd.concat([labels, features], axis=1)
result["id"] = result["id"].apply(lambda x: x.split("/")[-1][:-4]) 
result['id'] = result['id'].apply(pd.to_numeric)

joined = pd.merge(race_table, result, how='right', on="id")
print("dropping ", joined["gender"].isnull().sum(), "NaNs")
joined =  joined.dropna()
joined['ethnicity'].value_counts()

('dropping ', 61977, 'NaNs')


caucasian     578649
hispanic      243266
southasian     66765
eastasian      36009
african         5282
Name: ethnicity, dtype: int64

## Dropping a lot of rows to create a uniform distribution

In [23]:
cauc = joined[joined['ethnicity'] == 'caucasian'].sample(538649)
hisp = joined[joined['ethnicity'] == 'hispanic'].sample(203266)
east = joined[joined['ethnicity'] == 'eastasian'].sample(0)
sout = joined[joined['ethnicity'] == 'southasian'].sample(26765)
afri = joined[joined['ethnicity'] == 'african'].sample(5282)

In [24]:
uniform = joined.drop(cauc.index, axis=0)
uniform = uniform.drop(hisp.index, axis=0)
uniform = uniform.drop(sout.index, axis=0)
uniform = uniform.drop(east.index, axis=0)
uniform = uniform.drop(afri.index, axis=0)
uniform['ethnicity'].value_counts()

southasian    40000
caucasian     40000
hispanic      40000
eastasian     36009
Name: ethnicity, dtype: int64

In [25]:
uniform.to_csv('../Data/uniform_table.csv')

In [26]:
uniform = pd.read_csv('../Data/uniform_table.csv')

In [27]:
binary = uniform['ethnicity'].isin(['eastasian']).astype(int)
uniform['eastasian'] = binary
uniform

,Unnamed: 0,Unnamed: 0.1,id,gender,ethnicity,drop,0,1,2,3,...,119,120,121,122,123,124,125,126,127,eastasian
0,15,336.0,6.745248e+10,0.0,southasian,1,-0.017973,0.234318,0.045868,0.100706,...,0.054241,0.115467,-0.042813,-0.031940,0.022375,0.185700,-0.012256,0.154224,0.089849,0
1,26,660.0,8.719630e+09,1.0,hispanic,1,0.089673,0.067527,-0.057139,0.028362,...,0.135153,0.080423,-0.154849,-0.033321,-0.025678,-0.026101,0.076568,0.040354,0.038686,0
2,28,726.0,7.295041e+10,1.0,southasian,1,0.048146,-0.038316,-0.110152,0.034591,...,0.168158,-0.072299,-0.137659,-0.031673,-0.034293,-0.191415,0.148576,-0.120498,0.015287,0
3,30,836.0,9.724301e+10,2.0,caucasian,1,-0.047999,0.090907,0.126638,-0.057377,...,0.014449,0.055942,0.030899,-0.011251,0.041431,0.138157,-0.013602,0.111036,0.123017,0
4,48,1705.0,2.838980e+10,0.0,southasian,1,0.028755,0.172809,-0.119323,-0.098589,...,-0.047848,0.068467,-0.165270,0.058887,-0.050953,0.060163,-0.032356,0.040983,-0.048044,0
5,53,1896.0,4.221302e+10,0.0,southasian,1,-0.071434,0.112948,0.005628,0.129046,...,0.139625,0.119657,-0.061923,-0.066254,0.051291,0.084507,0.088415,0.093292,0.060611,0
6,59,2107.0,1.838855e+10,0.0,southasian,1,0.057718,0.131495,-0.068548,0.042353,...,0.079239,-0.082707,0.149726,0.123912,0.044913,0.077077,-0.048466,0.044201,-0.083011,0
7,75,2528.0,2.490026e+10,0.0,southasian,1,0.033196,0.127582,0.054308,0.024066,...,0.113136,0.163637,-0.076061,0.028769,0.057129,0.037534,-0.022008,0.137818,-0.017705,0
8,83,2677.0,4.805191e+10,1.0,caucasian,1,-0.076977,0.236791,-0.036860,-0.125795,...,0.065918,0.056030,0.119242,0.141702,-0.089253,-0.073802,-0.054072,0.005442,0.060238,0
9,85,2818.0,3.832058e+10,1.0,hispanic,1,0.005861,-0.037129,-0.006280,-0.092772,...,0.223428,-0.008948,-0.049182,-0.064812,0.020463,-0.223939,0.063452,-0.046506,0.043658,0


In [28]:
uniform = uniform.drop(['Unnamed: 0','Unnamed: 0.1', 'drop'], axis=1)
uniform.to_csv('../Data/uniform_table.csv')

In [33]:
print(uniform['id'][1])
uniform.loc[1]

8719629969.0


id           8.71963e+09
gender                 1
ethnicity       hispanic
0              0.0896732
1              0.0675273
2             -0.0571387
3              0.0283618
4              0.0486096
5               0.210486
6             -0.0117577
7             -0.0588593
8              -0.120097
9              0.0999808
10                0.1561
11             0.0705445
12             0.0286759
13            -0.0419754
14             0.0166638
15             0.0346226
16              0.049537
17              0.205269
18            -0.0113917
19            -0.0342632
20             0.0578529
21             0.0852498
22             0.0728323
23              0.223618
24             0.0231994
25            -0.0450945
26              0.111116
                ...     
99            -0.0011769
100            0.0366834
101              0.13723
102            0.0415785
103            0.0515222
104          -0.00893283
105           -0.0060107
106           -0.0586234
107           -0.0627475
